In [8]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.metrics import categorical_crossentropy
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
train_path = 'data/signs/train'
test_path = 'data/signs/test'
valid_path = 'data/signs/valid'

In [3]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=train_path, target_size=(224,224), batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=valid_path, target_size=(224,224), batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=test_path, target_size=(224,224), batch_size=10, shuffle=False)

Found 9993 images belonging to 36 classes.
Found 1080 images belonging to 36 classes.
Found 180 images belonging to 36 classes.


In [15]:
model1 = Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same', input_shape=(224, 224, 3)),
    MaxPool2D(pool_size=(2,2), strides=2),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(pool_size=(2,2), strides=2),
    Flatten(),
    Dense(units=36, activation='softmax')
])

In [16]:
model1.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
model1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 64)      18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 200704)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 36)                7225380   
Total params: 7,244,772
Trainable params: 7,244,772
Non-trainable params: 0
____________________________________________

In [18]:
model1.fit(x=train_batches, validation_data=valid_batches, epochs=10, verbose=2)

Epoch 1/10
1000/1000 - 357s - loss: 2.4590 - accuracy: 0.3399 - val_loss: 1.8720 - val_accuracy: 0.4852
Epoch 2/10
1000/1000 - 539s - loss: 0.5810 - accuracy: 0.8455 - val_loss: 2.0637 - val_accuracy: 0.4750
Epoch 3/10
1000/1000 - 587s - loss: 0.0326 - accuracy: 0.9940 - val_loss: 2.9495 - val_accuracy: 0.4704
Epoch 4/10
1000/1000 - 342s - loss: 0.0049 - accuracy: 0.9989 - val_loss: 3.1277 - val_accuracy: 0.4824
Epoch 5/10
1000/1000 - 347s - loss: 0.0110 - accuracy: 0.9972 - val_loss: 3.7976 - val_accuracy: 0.4093
Epoch 6/10
1000/1000 - 363s - loss: 0.0056 - accuracy: 0.9986 - val_loss: 5.1222 - val_accuracy: 0.3639
Epoch 7/10
1000/1000 - 336s - loss: 0.0232 - accuracy: 0.9933 - val_loss: 4.4366 - val_accuracy: 0.4222
Epoch 8/10
1000/1000 - 344s - loss: 0.0071 - accuracy: 0.9979 - val_loss: 4.8200 - val_accuracy: 0.4278
Epoch 9/10
1000/1000 - 362s - loss: 0.0147 - accuracy: 0.9953 - val_loss: 5.2495 - val_accuracy: 0.3870
Epoch 10/10
1000/1000 - 385s - loss: 0.0075 - accuracy: 0.9978 -

In [19]:
test_batches.classes

array([ 0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  3,  3,
        3,  3,  3,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  6,  6,  6,  6,
        6,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9, 10,
       10, 10, 10, 10, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 13, 13, 13,
       13, 13, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16,
       17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 20, 20,
       20, 20, 20, 21, 21, 21, 21, 21, 22, 22, 22, 22, 22, 23, 23, 23, 23,
       23, 24, 24, 24, 24, 24, 25, 25, 25, 25, 25, 26, 26, 26, 26, 26, 27,
       27, 27, 27, 27, 28, 28, 28, 28, 28, 29, 29, 29, 29, 29, 30, 30, 30,
       30, 30, 31, 31, 31, 31, 31, 32, 32, 32, 32, 32, 33, 33, 33, 33, 33,
       34, 34, 34, 34, 34, 35, 35, 35, 35, 35])

In [21]:
predictions = model1.predict(x=test_batches, verbose=0)

In [22]:
cm = confusion_matrix(y_true=test_batches.classes, y_pred=np.argmax(predictions, axis=-1))

In [49]:
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [53]:
category = list(test_batches.class_indices.keys())

In [76]:
# Compute confusion matrix
cm = confusion_matrix(y_true=test_batches.classes, y_pred=np.argmax(predictions, axis=-1))
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cm, labels=category, title='Confusion matrix')
plt.show()

Confusion matrix, without normalization
